# Melon 주간 차트 크롤링

In [1]:
import requests
import pandas as pd

In [2]:
# 한빛 아카데미
url = 'https://www.hanbit.co.kr/academy/'
req = requests.get(url)
html = req.text
# html

In [3]:
# 멜론 주간차트
# 헤더에 User-Agent 값을 주어야만 데이터를 가져올 수 있음.
url = 'https://www.melon.com/chart/week/index.htm'
header = {'User-Agent':
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36"}
req = requests.get(url, headers=header)
html = req.text
# html

In [4]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, 'html.parser')
# soup

In [ ]:
# #lst50 > td:nth-child(6) > div > div > div.ellipsis.rank02 > a (copy selector)
# //*[@id="lst50"]/td[6]/div/div/div[2]/a (copy Xpath)

In [5]:
trs = soup.select('#lst50')
len(trs)

50

In [6]:
tr = trs[0]
tr

<tr class="lst50" data-song-no="33507137" id="lst50">
<td><div class="wrap t_right"><input class="input_check" name="input_check" title="Butter 곡 선택" type="checkbox" value="33507137"/></div></td>
<td><div class="wrap t_center"><span class="rank">1</span><span class="none">위</span></div></td>
<!-- 차트순위 추가 -->
<td><div class="wrap">
<span class="rank_wrap" title="14단계 상승">
<span class="bullet_icons rank_up"><span class="none">단계 상승</span></span>
<span class="up">14</span>
</span>
</div></td>
<td><div class="wrap">
<a class="image_typeAll" href="javascript:melon.link.goAlbumDetail('10612483');" title="Butter">
<img alt="Butter - 페이지 이동" height="60" onerror="WEBPOCIMG.defaultAlbumImg(this);" src="https://cdnimg.melon.co.kr/cm2/album/images/106/12/483/10612483_20210521111412_500.jpg/melon/resize/120/quality/80/optimize" width="60">
<span class="bg_album_frame"></span>
</img></a>
</div></td>
<td><div class="wrap">
<a class="btn button_icons type03 song_info" href="javascript:melon.link.goSon

In [7]:
# 순위 찾기
rank = tr.select_one('.rank').get_text()  # string 또는 get.text()
rank = int(rank)
rank

1

In [8]:
# 곡명
title = tr.select_one('.ellipsis.rank01')
title = title.find('a').get_text()
title

'Butter'

In [9]:
# aritist
artist = tr.select_one('.ellipsis.rank02')
artist = artist.find('a').get_text()
artist

'방탄소년단'

In [10]:
#n 앨범
album = tr.select_one('.ellipsis.rank03')
album = album.find('a').string
album

'Butter'

In [27]:
## 50곡에 대해서 데이터 프레임 만들기.
rank_list, title_list, artist_list, album_list = [],[],[],[]
for tr in trs:
    rank = tr.select_one('.rank').get_text()  # string 또는 get.text()
    rank = int(rank)
    title = tr.select_one('.ellipsis.rank01')
    title = title.find('a').get_text()
    artist = tr.select_one('.ellipsis.rank02')
    artist = artist.find('a').get_text()
    album = tr.select_one('.ellipsis.rank03')
    album = album.find('a').string

    rank_list.append(rank)
    title_list.append(title)
    artist_list.append(artist)
    album_list.append(album)

In [30]:
df = pd.DataFrame({
    '순위': rank_list,
    '곡명': title_list,
    '가수': artist_list,
    '앨범': album_list
})
df.head()

,순위,곡명,가수,앨범
0,1,Butter,방탄소년단,Butter
1,2,Dun Dun Dance,오마이걸 (OH MY GIRL),Dear OHMYGIRL
2,3,Peaches (Feat. Daniel Caesar & Giveon),Justin Bieber,Justice
3,4,헤픈 우연,헤이즈 (Heize),HAPPEN
4,5,롤린 (Rollin'),브레이브걸스,Rollin'


# 이미지 속성값 가져오기

In [14]:
image = trs[0].select_one('.image_typeAll').find('img')
image = image['src']
image

'https://cdnimg.melon.co.kr/cm2/album/images/106/12/483/10612483_20210521111412_500.jpg/melon/resize/120/quality/80/optimize'

# 100곡 모두 가져오기

In [15]:
trs = soup.select('#lst50')
trs2 = soup.select('#lst100')
trs.extend(trs2)
len(trs)

100

In [32]:
## 50곡에 대해서 데이터 프레임 만들기.
rank_list, title_list, artist_list,album_list, image_list = [],[],[],[],[]
for tr in trs:
    rank = tr.select_one('.rank').get_text()  # string 또는 get.text()
    rank = int(rank)
    title = tr.select_one('.ellipsis.rank01')
    title = title.find('a').get_text()
    artist = tr.select_one('.ellipsis.rank02')
    artist = artist.find('a').get_text()
    album = tr.select_one('.ellipsis.rank03')
    album = album.find('a').string
    image = trs[0].select_one('.image_typeAll').find('img')
    image = image['src']

    rank_list.append(rank)
    title_list.append(title)
    artist_list.append(artist)
    album_list.append(album)
    image_list.append(image)

In [33]:
df = pd.DataFrame({
    '순위': rank_list,
    '곡명': title_list,
    '가수': artist_list,
    '앨범': album_list,
    '이미지': image_list
})
df.head()

,순위,곡명,가수,앨범,이미지
0,1,Butter,방탄소년단,Butter,https://cdnimg.melon.co.kr/cm2/album/images/10...
1,2,Dun Dun Dance,오마이걸 (OH MY GIRL),Dear OHMYGIRL,https://cdnimg.melon.co.kr/cm2/album/images/10...
2,3,Peaches (Feat. Daniel Caesar & Giveon),Justin Bieber,Justice,https://cdnimg.melon.co.kr/cm2/album/images/10...
3,4,헤픈 우연,헤이즈 (Heize),HAPPEN,https://cdnimg.melon.co.kr/cm2/album/images/10...
4,5,롤린 (Rollin'),브레이브걸스,Rollin',https://cdnimg.melon.co.kr/cm2/album/images/10...


# 파일에 저장하기

In [34]:
period = soup.select_one('.yyyymmdd').get_text().strip()  # strip 불필요한거 제거
period

'2021.05.24 ~ 2021.05.30'

In [35]:
filename = f'melon({period}).csv'
df.to_csv(filename, index=False, sep=',', encoding='utf8')